In [1]:
!pip install -U faiss-cpu langchain langchain-community sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached aiohttp-3.12.13-cp312-cp312-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached transformers-4.53.0-py3-none-any.whl.metadata (39 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached huggingface_hub-0.33.1-py3-none-any.whl.metadata (14 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached langchain_community-0.3.26-py3-none-any.whl (2.5 MB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
Using cached aiohttp-3.12.13-cp312-cp312-win_amd64.whl (447 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Usi

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [3]:
pip install langchain openai faiss-cpu pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

In [7]:
df = pd.read_csv("amazon-products.csv")

In [8]:
df.head()

,timestamp,title,seller_name,brand,description,initial_price,final_price,currency,availability,reviews_count,...,root_bs_category,bs_category,bs_rank,badge,subcategory_rank,amazon_choice,images,product_details,prices_breakdown,country_of_origin
0,2023-08-08 00:00:00.000,Saucony Men's Kinvara 13 Running Shoe,Orv███tor███,Saucony,"When it comes to lightweight speed, nothing cr...",NaN,"""57.79""",USD,In Stock,702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-09 00:00:00.000,Kishigo Premium Black Series Heavy Duty Unisex...,Ama███.co███,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,NaN,"""28.5""",USD,In Stock,916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-04 00:00:00.000,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,Twi███uxe███,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,"""49.99""","""33.99""",USD,In Stock,3178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:00:00.000,Accutire MS-4021B Digital Tire Pressure Gauge ...,Cit███ran███Dir██████,Accutire,About this item Heavy duty construction and ru...,1.795000000000000e+01,1.795000000000000e+01,USD,In Stock,8034,...,Automotive,Tire Repair Tools,50.0,NaN,"[{""subcategory_name"":""Automotive"",""subcategory...",False,NaN,NaN,NaN,NaN
4,2024-01-16 00:00:00.000,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,PRA███ EN███PRI███,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,"""1299""","""799""",INR,In stock,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.shape

(1000, 55)

In [10]:
df = df[df['title'].notnull() & df['description'].notnull()].copy()

In [11]:
df.shape

(998, 55)

In [12]:
df.head()

,timestamp,title,seller_name,brand,description,initial_price,final_price,currency,availability,reviews_count,...,root_bs_category,bs_category,bs_rank,badge,subcategory_rank,amazon_choice,images,product_details,prices_breakdown,country_of_origin
0,2023-08-08 00:00:00.000,Saucony Men's Kinvara 13 Running Shoe,Orv███tor███,Saucony,"When it comes to lightweight speed, nothing cr...",NaN,"""57.79""",USD,In Stock,702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-09 00:00:00.000,Kishigo Premium Black Series Heavy Duty Unisex...,Ama███.co███,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,NaN,"""28.5""",USD,In Stock,916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-04 00:00:00.000,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,Twi███uxe███,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,"""49.99""","""33.99""",USD,In Stock,3178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:00:00.000,Accutire MS-4021B Digital Tire Pressure Gauge ...,Cit███ran███Dir██████,Accutire,About this item Heavy duty construction and ru...,1.795000000000000e+01,1.795000000000000e+01,USD,In Stock,8034,...,Automotive,Tire Repair Tools,50.0,NaN,"[{""subcategory_name"":""Automotive"",""subcategory...",False,NaN,NaN,NaN,NaN
4,2024-01-16 00:00:00.000,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,PRA███ EN███PRI███,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,"""1299""","""799""",INR,In stock,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df["text"] = df["title"] + ". " + df["description"]
df["text"]

0      Saucony Men's Kinvara 13 Running Shoe. When it...
1      Kishigo Premium Black Series Heavy Duty Unisex...
2      TWINSLUXES Solar Post Cap Lights Outdoor - Wat...
3      Accutire MS-4021B Digital Tire Pressure Gauge ...
4      SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...
                             ...                        
995    Madison Park Amherst Faux Silk Comforter Set-C...
996    BIC Gel-Ocity Gel Pens, Medium Point Retractab...
997    Elmer's E842L Carpenter's Interior Wood Filler...
998    Office Chair Mat for Carpet, 30” x 48” Transpa...
999    Bavilk Kids Bike Helmet,Boys Girls Helmet Mult...
Name: text, Length: 998, dtype: object

In [14]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\asati\AppData\Local\Temp\ipykernel_6220\2537902316.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\asati\OneDrive\Desktop\Product_Similarity_Search\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\asati\OneDrive\Desktop\Product_Similarity_Search\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to e

In [15]:
documents = [Document(page_content=text) for text in df["text"].tolist()]

In [16]:
vectorstore = FAISS.from_documents(documents, embedding_model)

In [17]:
query = "Saucony Men's Kinvara 13 Running Shoe"
results = vectorstore.similarity_search(query, k=3)
for i, res in enumerate(results):
    print(f"{i+1}. {res.page_content}")

1. Saucony Men's Kinvara 13 Running Shoe. When it comes to lightweight speed, nothing crushes the competition like the Kinvara. And this just so happens to be our lightest one yet. With more speed contouring and its signature flexible feel, it pushes you forward without holding anything back. These are the shoes you’ll do big things in.
2. adidas Men's Ultraboost Personal Best Running Shoe. Men's shoes - low (non football).
3. Skechers Women's Max Cushioning Elite Sneaker. Haptic printed max cushion running shoe.


In [18]:
vectorstore.save_local("product_index")